In [2]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM, TimeDistributed
# from keras.layers import Concatenate, Flatten
# from keras.layers import GRU, Conv2D, MaxPooling2D
# from keras.layers import Input, Reshape, Dot, Add
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
# import keras
# import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
# import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
# tf.set_random_seed(seed_val)

c:\users\amany\anaconda2\envs\tf2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
words = read_file(filename='data/news.txt')
# unkown_word = "<unk>"
# words = [unkown_word] + words
xvocab, xword2int, xint2word = build_vocab(words)

words, word2freq = min_count_threshold(words)
vocab, word2int, int2word = build_vocab(words)
print(len(vocab))
# word2freq = get_frequency(words, word2int, int2word)
unigrams = [word2freq[int2word[i]] for i in range(len(word2int))]

char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 100
skip_window = 5

58383


In [4]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    embeddings = np.zeros((vocab_size, embed_size), dtype=np.float64)
#     print(embeddings.shape)
    wordint = {}
    for i in range(1, vocab_size+1):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
#             if word in word2int:
            wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
            embeddings[len(wordint)] = wordvec
            wordint[word] = len(wordint)
        except Exception as e:
#             print(lines[i])
#             print(e)
            pass
#     print(len(wordint))
    return embeddings, wordint

In [4]:
def collectArabicEval(result):
    nsemantic = 0
    nsynatctic = 0
    for nrr in result:
        if 'total' in nrr:
            continue
        if "gram" in nrr:
            nsynatctic += result[nrr]/6
        else:
            nsemantic += result[nrr]/9
    return nsynatctic, nsemantic

In [ ]:
csv.close()


In [ ]:
import os
folder = 'results/'
file_list = os.listdir(folder)[2:]
row = "{0},{1},{2},{3},{3},{4},{5},{6},{7},{8}"
csv = open('sg4.csv', mode='a')
csv.write(row.format('data', 'size', 'model', 'embed_size', 'win_size', 'norm-syn', 'norm-sem', 'unnorm-syn', 'unnorm-sem'))
csv.write('\n')
csv.close()
for file in file_list:
    line = open(folder + file).readline()
    vals = file.split('_')
    print(file)
    data, model, embed_size, win_size, _ = vals

    name = 'am'
    size = 0
    corpus =  'data/corpus/am16.txt'
    question = 'data/newan2.txt'
    if 'am' in data:
        name = data[:2]
        size = data[2:]
    if 'ar' in data:
        corpus =  'data/corpus/20'
        name = data[:2]
        size = data[2:]
        question = 'data/corpus/ar_questions.txt'
    if 'abj' in data:
        name = 'abj'
        size = data[3:]
        corpus =  'data/corpus/abj-am-all.txt'
        question = 'data/abj-am-analogy.txt'
    if 'alpha' in data:
        name = 'alpha'
        size = data[5:]
        corpus =  'data/corpus/alpha-all.txt'
        question = 'data/alpha-am-analogy.txt'
    seq_emb = parseVec(folder + file, ' ')
#         text = open('data/grid/' + file).read()
#         text = text.replace("'", '"')
    emb = seq_emb[0]
    wordint = seq_emb[1]
    norm_res = evaluate(wordint, normalize(emb), embed_size=emb.shape[1], corpus=corpus, analogy=question)
    unorm_res = evaluate(wordint, emb, embed_size=emb.shape[1], corpus=corpus, analogy=question)
    nsyn, nsem = collectArabicEval(norm_res)
    usyn, usem = collectArabicEval(unorm_res)
#     nsyn, nsem = norm_res['syntactic'], norm_res['semantic']
#     usyn, usem = unorm_res['syntactic'], unorm_res['semantic']
    csv = open('sg4.csv', mode='a')
    csv.write(row.format(name, size, model, embed_size, win_size, nsyn, nsem, usyn, usem))
    csv.write('\n')
    csv.close()

In [5]:
seq_emb = parseVec("results/vec.txt", ' ')

In [10]:
emb = seq_emb[0]
wordint = seq_emb[1]
print(len(wordint))
print(evaluate(wordint, normalize(emb), embed_size=emb.shape[1]))
print(evaluate(wordint, emb, embed_size=emb.shape[1]))

226397
{'syntactic': 2.5700934579439254, 'semantic': 0.36900369003690037, 'total': 1.7167381974248928}
{'syntactic': 0.0, 'semantic': 0.0, 'total': 0.0}


In [ ]:
print(emb[0])

In [ ]:
emb2D = np.zeros((seq_emb.shape[0], seq_emb.shape[1] * sem_emb.shape[1]))
for i in range(seq_emb.shape[0]):
    seq = seq_emb[i]
    sem = sem_emb[i]
    vec = np.outer(seq, sem).flatten()
    emb2D[i] = vec

In [ ]:
lines = open('data/newan2.txt', encoding='utf-8').readlines()
emb = emb2D
correct = 0
for line in lines:
    if ':' in line:
        continue
    try:
        line = [x.strip() for x in line[:-1].split(' ')]
        vs = [emb[word2int[w]] for w in line]
        v = vs[1] - vs[0] + vs[2]
        sims = emb.dot(v)
        sim_with_word = [(word, sims[word2int[word]])
                             for word in word2int.keys()]
        sorted_sim = sorted(sim_with_word, key=lambda t: t[1], reverse=True)[:4]
#         print(type(sorted_sim[0][1]), sorted_sim[0][1])
        result = [pred[0] for pred in sorted_sim]
        if not np.isnan(sorted_sim[0][1]):
#             print(result, line[-1])
            if line[-1] in result:
#                 print(result, line[-1])
                correct += 1
    except:
        pass
print(correct * 100.0 / (len(lines) - 2))

In [ ]:
# util = Utils(word2int, int2word, emb2D)
# lines = open('data/newan2.txt', encoding='utf-8').readlines()
# acc = np.array([0, 0])
# total = np.array([0, 0])
# ind = 0
# for line in lines[1:]:
#     if ':' in line[:-1]:
#         ind = 1
#         continue
#     ws = line[:-1].split()
#     result = util.solve(ws)
#     if result is None:
#         continue
#     if ws[-1] in result:
#         acc[ind] += 1
#     total[ind] += 1
# print(acc * 100 / total)

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# e = normalize(cew)
# # e = cew
# for line in lines:
# #     ws = "ልጅቱ ልጁ ወይዘሮ አቶ".split(' ')
#     if ":" in line:
#         continue
#     try:
#         ws = line[:-1].split(' ')
#         v = normalize(vec(ws[1], e) - vec(ws[0], e) + vec(ws[2], e))
#         V = np.dot(e, v)
#         sort = list(sorted(enumerate(V[1:]), key=lambda x: x[1], reverse=True))
#         so = [int2word[index[0]] for index in sort[:4]]
#         if ws[-1] in so:
#             print(so, ws[-1])
# #         break
#     except:
#         pass

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             words.append(ws)
# ws = []
# for wl in words:
#     s = ' '.join(wl)
#     ws.append(s)
# ws = '\n'.join(ws)
# open('data/newan.txt', mode='w', encoding='utf-8').write(ws)

        
        

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# lns = {}
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             ln = ' '.join(ws)
#             if ln not in lns:
#                 lns[ln] = len(lns)
#             ls = ' '.join(ws[2:4] + ws[0:2])
#             if ln not in lns:
#                 lns[ln] = len(lns)
# ws = '\n'.join(lns.keys())
# open('data/newan2.txt', mode='w', encoding='utf-8').write(ws)